In [1]:
from PIL import Image
from random import shuffle
import numpy as np
import tensorflow as tf
import glob
import cv2

In [2]:
data_path='ALL_IDB2/ALL_IDB2/img'
height = 256
width = 256

def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (width,height ), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = img.astype(np.float32)
    return img

def label_img(image_path):
    if image_path[28] == "0":
        return [1, 0]
    else:
        return [0, 1]

def save_image(image, filename):
    image = np.clip(image, 0.0, 255.0)
    image = image.astype(np.uint8)
    img = Image.fromarray(image)
    print(img.show())
    
def load_image2(image_path):
    #print(image_path)
    img = Image.open(image_path)
    img = img.resize((width, height))
    img
    image_arr = np.asarray(img, dtype='float32')
    image_arr = np.expand_dims(image_arr, axis=0)
    
    return img 
#xx = load_image('ALL_IDB2/ALL_IDB2/img\\Im004_1.tif')
#save_image(xx,'yt')

In [3]:
#Collect all the image addresses
image_address = []
image_address.extend(glob.glob(data_path+'/*.tif'))

In [4]:
#Prep training data.
def create_training_data():
    data = []
    for addr in image_address:
        label = label_img(addr)
        image = load_image(addr)
        data.append([np.array(image),np.array(label)])
    shuffle(data)
    
    training_data = data[:210]
    testing_data = data[210:]
    
    print(len(training_data))
    print(len(testing_data))
    
    return training_data,testing_data

#get data and labels from the created data.
def create_req(x):
    x_data = []
    x_label = []
    for i in range(len(x)):
        x_data.append(x[i][0])
        x_label.append(x[i][1])
    
    return x_data,x_label

In [5]:
training_data, testing_data = create_training_data()

train_x, train_y = create_req(training_data)
test_x, test_y = create_req(testing_data)


210
50


In [6]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')

out_num = 2

x = tf.placeholder('float')
y = tf.placeholder('float')

def conv_net(x):
    x = tf.reshape(x, [-1, 256, 256, 1])
    W_conv1 = weight_variable([32, 32, 1, 16])
    W_conv2 = weight_variable([32, 32, 16, 32])
    #W_conv3 = weight_variable([32, 32, 32, 32])
    W_fc1 = weight_variable([16*16*32, 512])
    W_out = weight_variable([512, out_num])
    
    b_conv1 = bias_variable([16])
    b_conv2 = bias_variable([32])
    #b_conv3 = bias_variable([32])
    b_fc1 = bias_variable([512])
    b_out = bias_variable([out_num])
    
    conv1 = maxpool( tf.nn.relu(conv2d(x,W_conv1)+b_conv1) )
    conv2 = maxpool( tf.nn.relu(conv2d(conv1,W_conv2)+b_conv2) )
    #conv3 = maxpool( tf.nn.relu(conv2d(conv2,W_conv3)+b_conv3) )
    
    fc1 = tf.reshape(conv2,[-1, 16*16*32 ])
    fc1 = tf.nn.relu(tf.matmul(fc1,W_fc1)+b_fc1)
    fc1 = tf.nn.dropout(fc1, 0.8)
    
    output = (tf.matmul(fc1,W_out))
    
    return output

In [ ]:
def train_network(x):
    
    predictions = conv_net(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = predictions,labels = y))
    #learning_rate = 
    optimizer = tf.train.AdamOptimizer(0.00001).minimize(cost)
    
    hm_epoches = 80
    batch_size = 5
    
    loss_arr = []
    acc_arr = []
    
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
    sess.run(tf.global_variables_initializer())
    
    print("Started Training ----")
        
    for epoch in range(hm_epoches):
        epoch_loss = 0
        i = 0
        
        batch_x = []
        batch_y = []
        
        while i< len(train_x):
            start = i;
            end = i+batch_size
            batch_x = train_x[start:end]
            batch_y = train_y[start:end]
                
            _,c = sess.run([optimizer, cost], feed_dict={x:batch_x , y:batch_y})
            epoch_loss+=c
            print('Batch',i,'Epoch',epoch,'/',hm_epoches,'loss:',c)
            i=end
        
        correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        acc_test = 0
        if epoch%5 == 0:
            acc = accuracy.eval(session=sess, feed_dict={x:batch_x, y:batch_y})
            #acc_test = accuracy.eval(session=sess, feed_dict={x:test_x, y:test_y})
            print('Epoch',epoch,'Loss:',epoch_loss,'Train Accuracy:',acc,'Test : ',acc_test)
        else:
            print('Epoch',epoch,'Loss:',epoch_loss)
        #acc1 = accuracy.eval({x:train_x, y:train_y})
        
        #acc = accuracy.eval(session=sess, feed_dict={x:test_x, y:test_y})
        loss_arr.append(epoch_loss)
        #acc_arr.append(acc_test)
        
        #print('Epoch',epoch,'Loss:',epoch_loss,' Test Accuracy:',acc)
        
    return loss_arr, sess
#Accuracy: 0.666667   learning rate=0.0005   epoches = 10
#Accuracy: 0.833333   learning rate=0.0002   epoches = 20
#Accuracy: 0.833333   learning rate=0.0002   epoches = 20    Dropout = True 0.8 --WASTE

In [ ]:
loss = []
acc = []

loss, sess = train_network(x)

In [ ]:
predictions = conv_net(x)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = predictions,labels = y))

#learning_rate = 
optimizer = tf.train.AdamOptimizer(0.0001).minimize(cost)    
hm_epoches = 50
batch_size = 5
    
loss_arr = []
acc_arr = []
    
print("Started Training ----")
   
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
    
sess.run(tf.global_variables_initializer())
        
for epoch in range(hm_epoches):
    epoch_loss = 0
    i = 0
        
    batch_x = []
    batch_y = []
        
    while i< len(train_x):
        start = i;
        end = i+batch_size
        batch_x = train_x[start:end]
        batch_y = train_y[start:end]
            
        _,c = sess.run([optimizer, cost], feed_dict={x:batch_x , y:batch_y})
        epoch_loss+=c
        print('Batch',i,'Epoch',epoch,'/',hm_epoches,'loss:',c)
        i=end
        
    correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    if epoch%5 == 0:
        acc = accuracy.eval(session=sess, feed_dict={x:batch_x, y:batch_y})
        print('Epoch',epoch,'Loss:',epoch_loss,'Train Accuracy:',acc)
    else:
        print('Epoch',epoch,'Loss:',epoch_loss)
        
    #acc1 = accuracy.eval({x:train_x, y:train_y})     
    #acc = accuracy.eval(session=sess, feed_dict={x:test_x, y:test_y})
    loss_arr.append(epoch_loss)
    #acc_arr.append(acc)
    #print('Epoch',epoch,'Loss:',epoch_loss,' Test Accuracy:',acc)

Started Training ----


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#plt.figure(1)
x = range(len(loss))
#print(x)
plt.plot(x, loss)

In [ ]:
pred = conv_net(x)
correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

test_er = accuracy.eval(session=sess, feed_dict={x:test_x, y:test_y})
print(test_er)
#xx = range(len(loss))
#plt.plot(xx,loss)

In [ ]:
#for i in range(len(acc)):
#    acc[i] = acc[i]*100
#print(acc)
plt.plot(x, acc2)

In [ ]:
plt.plot(xx, acc)